In [9]:
# Primerament importem totes les llibreries necessaries per completar el projecte.
import os
import pandas as pd
import requests
import argparse
import csv
import pathlib 
import time
from datetime import datetime
from bs4 import BeautifulSoup
import json

In [10]:
# Carreguem el fitxer de configuració a la variable config:
with open("configuration.json", "r") as configFile:
    config = json.load(configFile)

In [23]:
# Funcions per obtindre la llista del indexs del Ibex 35 de la borsa de Barcelona

# Get Header
def getHeaders():   
    return config['header']


# Get table Page        
def getPageTable():        
    table = None
    try:
        # Data for post form    
        data = {'punto':'indice'}
        page = requests.post(config['URLs']['base'] + config['URLs']['ref'],timeout=10)
        soup = BeautifulSoup(page.text, "html.parser")
        table = soup.find('table', attrs={'id': 'ctl00_Contenido_tblAcciones'})
    except requests.exceptions.Timeout:
        print("Timeout a la càrrega de la web")
    except:        
        print("Error inesperat")
    return table        

# Get Firm page and wait x seconds
def getFirmTablePage(UrlFirmPage):    
    table = None    
    try:
        t0 = time.time()
        # Data for post form            
        data = {'punto':'indice'}    
        page = requests.get(UrlFirmPage,timeout=10)                    
        soup = BeautifulSoup(page.text, "html.parser")                
        table = soup.find('table', attrs={'id': 'ctl00_Contenido_tblCapital'})                           
        # Wait time depending of response
        response_delay = time.time() - t0     
        time.sleep(10 * response_delay)
    except requests.exceptions.Timeout:
        print("-->Timeout a la càrrega de dada de la companyia")                
    except:                
        print("-->Error inesperat: " +  UrlFirmPage)
    return table

# Get the list of firm interesting values
def getFirmValue(firmPage): 
    value = None
    table = getFirmTablePage(config['URLs']['base']+firmPage)        
    if (table is not None):                
        nroFila=0
        rowNum=0
        for row in table.find_all("tr"):
            if rowNum==1:
                colNum=0                
                for cell in row.find_all('td'):
                    if colNum==1:
                        value = cell.text                                     
                        break;
                    colNum=colNum+1
            rowNum=rowNum+1       
    return value

# Get List with column page names
def getColNames():
    colList = []
    table = getPageTable()
    for tr in table.find_all("tr"):      
        # Cells Header
        cellsHead = tr.findAll('th')                
        if ((cellsHead is not None) and (len(cellsHead) > 0)):
            for cellHead in cellsHead:   
                colList.append(cellHead.text.strip())
    colList.append('Capitalización')
    return colList


# Get a pandas dataframe with table page values
def getDataframe():
    # Obtenció de la taula trobada a la pàgina web amb els valros a extreure
    table = getPageTable()    
    # Obtenció dels noms de totes els columnes
    colList = getColNames()
    # variable on es gaurdaran els valors de cada columna
    totalValues = []
    
    # Per cada fila de la taula
    for tr in table.find_all("tr"):              
        # Es busquen les capçaleres de les columnes
        cellsHead = tr.findAll('th')   
        # En cas de no tractar-se d'una capçalera:             
        if ((cellsHead is None) or (len(cellsHead) == 0)):
            # Es busquen totes les cel·les amb valors
            cells = tr.findAll('td')
            companyValues = []
            # Per cada cel·la amb valors, s'afegeix el valor a la llista de companyValues
            capitalization = None
            for cell in cells:
                a= cell.find('a')                  
                if (a is not None):                 
                    href = a.get('href')
                    if href[0] !='/':
                        href = '/' + href
                    capitalization = getFirmValue(href) 
                companyValues.append(cell.text.strip())
            companyValues.append(capitalization)
            # Després d'obtenir tots els valors, els guardo a la llista totalValues fent que finalment
            # a la variable totalValues es tingui una llista de valors per a cada fila de la taula que no sigui capçalera
            totalValues.append(companyValues)
    
    # Genero un diccionari on la clau és cadascún dels noms de la capçalera de la taula i el valor una llista buida
    dicValues = {}
    for colName in colList:
        dicValues[colName] = []

    # Genero un dataframe amb les capçaleres de la taula com columnes usant el diccionari creat
    dataframe = pd.DataFrame(data=dicValues)
    # Per cada fila de la taula en format llista:
    for list_row in totalValues:
        row = {}
        index = 0
        # Per cada columna (capçalera de la taula), afegeixo al diccionari row el nom de la
        # capçalera com a key i el valor de la llista_row pertinent com a value.
        for colName in colList:
            row[colName] = list_row[index]
            index += 1
        # Afegeixo el diccionari row com a una nova fila del dataframe
        dataframe = dataframe.append(row, ignore_index=True)
    # Finalment, retorno el dataframe
    return dataframe 


# Funció que transforma un dataframe de pandas en un fitxer CSV
def writeCSV(dataframe, fileName):
    # Converteixo el dataframe en format CSV i mostro per pantalla el nom del CSV generat
    dataframe.to_csv(fileName, index = False, header=True)
    print('New CSV file generated: ' + fileName)


# Funció que genera fitxers CSV amb les dades de la web cada 15 minuts fins que es tanca el mercat
def ScheduleDayFile():
    # En cas de que el mercat no estigui tancat:
    while not markedClosed():
        # Adquisició del dataframe, generació del nom que tindrà el fitxer CSV i generació d'aquest
        # amb la funció wreiteCSV.
        dataframe = getDataframe()
        fileName = "ListaIndices" + time.strftime('%Y-%m-%d_%H-%M')+ ".csv"
        writeCSV(dataframe, fileName)
        # Un cop creat i guardat el fixter CSV, ens esperem 15 minuts fins tronar a comprobar l'estat del mercat
        print("Waiting 15 minutes")
        time.sleep(60*15)        
    print("End of session: file generated: " + "ListaIndices" + time.strftime('%Y-%m-%d_%H-%M')+ ".csv")


# Retorna un boolean indcant si el mercat està tancat o no
def markedClosed():
    return (getDataframe()['Hora'].iloc[0] == 'Cierre')

    
# Save the Stock Exchange one week
def ScheduleWeekFiles():    
    scheduler = BlockingScheduler()    
    scheduler.add_job(job_function, 'cron', day_of_week='mon-fri', hour=9, minute=0)    
    scheduler.start()    
    

# Retorna un dataframe amb els noms de les empreses que formen part de l'Ibex 35 i de les quals tenim dades
def getIndex():
    return getDataframe()['Nombre']
    

In [16]:
print(getColNames())

['Nombre', 'Últ.', '% Dif.', 'Máx.', 'Mín.', 'Volumen', 'Efectivo (miles €)', 'Fecha', 'Hora', 'Capitalización']


In [17]:
print(list(getIndex()))

['ACCIONA', 'ACERINOX', 'ACS', 'AENA', 'ALMIRALL', 'AMADEUS', 'ARCELORMIT.', 'B.SANTANDER', 'BA.SABADELL', 'BANKINTER', 'BBVA', 'CAIXABANK', 'CELLNEX', 'CIE AUTOMOT.', 'ENAGAS', 'ENDESA', 'FERROVIAL', 'FLUIDRA', 'GRIFOLS CL.A', 'IAG', 'IBERDROLA', 'INDITEX', 'INDRA A', 'INM.COLONIAL', 'MAPFRE', 'MELIA HOTELS', 'MERLIN', 'NATURGY', 'PHARMA MAR', 'R.E.C.', 'REPSOL', 'SIEMENS GAME', 'SOLARIA', 'TELEFONICA', 'VISCOFAN']


In [24]:
 ScheduleDayFile()

New CSV file generated: ListaIndices2021-04-08_19-14.csv
End of session: file generated: ListaIndices2021-04-08_19-15.csv
